In [ ]:
%pip install opencv-python
%pip install pillow
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install tensorflow==2.10
%pip install keras

In [ ]:
# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 224, 224


In [2]:
train_data_dir = '/home/subhrajit/Desktop/EE604/Assignment 2/datathonindoml-2022/train/train'
validation_data_dir = '/home/subhrajit/Desktop/EE604/Assignment 2/datathonindoml-2022/validation/validation'
nb_train_samples =16000
nb_validation_samples = 900
epochs = 10
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)

In [ ]:
## CNN Model Design
# Hyperparameter tuning

model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) # 16
model.add(Activation('sigmoid'))  # softmax activation function

In [5]:
# CNN -> Forward Propagation and Backward Porpagation

model.compile(loss='binary_crossentropy', # sparse categorical cross entropy -> multiclass classification
			optimizer='rmsprop', # 'adam' optimizer
			metrics=['accuracy'])

# model.summary()

In [ ]:
# image -> 3D array
# Data Preprocessing
# Image -> Numpy Arrays (should be a correct input to CNN)

train_datagen = ImageDataGenerator(
	rescale=1 / 255, # (0->255) -> (0->1)
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary') # binary classification

validation_generator = test_datagen.flow_from_directory( # validation_data -> numpy array 3D -> . . . . -> [] -> CNN
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary') # multiclass classification

# Model Trained
# Labels not used ??

model.fit_generator( # fit -> multiclass classification # -> model  parameters tune so that it fits the data with minimum loss function
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)


In [7]:
model.save_weights('model_saved.h5')  # learnable parameters store

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np

from keras.models import load_model

# took 1 image -> predicted its class using model
model = load_model('model_saved.h5')

image = load_img('v_data/test/planes/5.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
print("Predicted Class (0 - Yes , 1- No): ", label[0][0])  

In [ ]:

import ast
import sys

import cv2
import matplotlib.pyplot as plt
import pandas as pd
import PIL.Image
import torch
import torch.nn.functional as F

# sys.path.insert(0, '..')
# from pytorch_image_classification import (
#     get_default_config,
#     create_model,
#     create_transform,
# )
# config = get_default_config()
# config.merge_from_file('../configs/imagenet/resnet18.yaml')
# device = torch.device(config.device)

# model = create_model(config)
# checkpoint = torch.load(
#     '../experiments/imagenet/resnet18/exp00/checkpoint_00090.pth')

# model.load_state_dict(checkpoint['model'])
# model.to(device)
# _ = model.eval()
# transform = create_transform(config, is_train=False)
# %%capture
# !wget https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt
# with open('imagenet1000_clsidx_to_labels.txt') as f:
#     index2label = ast.literal_eval(f.read())
# %%capture
# !wget https://images.pexels.com/photos/2071873/pexels-photo-2071873.jpeg
# image = cv2.imread('pexels-photo-2071873.jpeg')
# plt.imshow(image[:, :, ::-1])
# plt.show()

# data = transform(PIL.Image.fromarray(image))
# with torch.no_grad():
#     pred = model(data.unsqueeze(0).to(device))
# prob = F.softmax(pred, dim=1).cpu()

# scores, indices = prob.topk(k=5)
# scores = scores.numpy().ravel()

# indices = indices.numpy().ravel()
# names = [index2label[index] for index in indices]
# pd.DataFrame({'label': names, 'score': scores})
# label	score
# 0	Egyptian cat	0.836771
# 1	lynx, catamount	0.022285
# 2	tabby, tabby cat	0.020165
# 3	Siamese cat, Siamese	0.017015
# 4	tiger cat	0.007150
 